In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder

age, _, _ = desired_lulc()

lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select([f"classification_{year}" for year in config.range_1985_2020])
        .byte()
        .rename([str(year) for year in config.range_1985_2020]))

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()
method = 'eu'
export = False

panama = True
if panama:
    roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_na", "equals", "Panama").geometry()
    folder = "panama"
else:
    folder = method

In [6]:

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()

if method == 'eu':
    annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(config.roi)
    mature_mask = annual_changes.select('Dec2020').eq(1).selfMask()
    mature_biomass = biomass_raw.updateMask(mature_mask).rename("mature_biomass")

elif method == 'mapbiomas':
    mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()
    # keep only forests that are 1000m away from any non-forest area
    edge_detec = mature_mask.unmask(-1).zeroCrossing()
    distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
        .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
    distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
    mature_biomass = biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

sur_cover = mature_mask.unmask(0).focalMean(radius = 1000, units = "meters").float().rename("sur_cover")

if export:
    export_image(mature_biomass, "mature_biomass", folder = method, crsTransform = proj['transform'])
    # export_image(mature_biomass, "mature_biomass_500", folder = method, scale = 500)
    export_image(sur_cover, "sur_cover", folder = method, scale = 30).updateMask(age)

map=geemap.Map()
map.addLayer(mature_biomass, {"min": 0, "max": 200}, "mature_biomass")
map

KeyboardInterrupt: 

In [ ]:
mature_biomass = ee.Image(f"{data_folder}/{method}/mature_biomass_500")

biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
mature_biomass = mature_biomass.clip(amaz_atla_pant)

mature_biomass_vector = mature_biomass.sample(region = mature_biomass.geometry(), scale = 1000, geometries = True)

secondary = ee.Image(f"{data_folder}/{method}/secondary")
secondary = secondary.select("age")



# change radius depending on data resolution and patterns
distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 10000)\
    .toInt().updateMask(secondary)

def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

categorical = ee.Image(f"{data_folder}/categorical")
biome_mask = categorical.select('biome').eq(1) \
.Or(categorical.select('biome').eq(4)) \
.Or(categorical.select('biome').eq(6))

distance_to_nearest_mature = distance_to_nearest_mature \
    .addBands(biome_mask)

distances_vector = distance_to_nearest_mature.stratifiedSample(
    numPoints = 50000, classBand = "biome", region = roi, geometries = True
)

distances_vector_buffer = distances_vector.map(buffer_distance)

nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                        crs = proj['crs'], crsTransform = proj['transform'])


# change geometries from polygons to points
nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

# Convert the FeatureCollection to an image
nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
    properties = ['first'],
    reducer = ee.Reducer.first()
).toInt16().rename('nearest_mature_biomass')

export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image", \
                folder = method, crsTransform = proj['transform'])